In [ ]:
import datasets

datasets.load_dataset("json", data_files="NEW_data_collection/data/raw_scraper_output.json")



In [3]:
import json

# load in json
with open("NEW_data_collection/data/raw_scraper_output.json") as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
with open("NEW_data_collection/data/raw_scraper_output.json", "r") as f:
    data = f.read()

In [ ]:
import pandas as pd
import json
import re
import os

gpt35 = pd.read_csv("NEW_data_collection/data/gpt-3.5-data-qa.csv")
gpt4 = pd.read_csv("NEW_data_collection/data/gpt-4-data-qa.csv")

In [ ]:
idx = 15
print("GPT-3.5 URL: "+gpt35['url'][idx])
print("GPT-4 URL: "+gpt4['url'][idx])
print("Context:\n"+gpt35['data'][idx])
print("GPT-3.5:\n"+json.dumps(json.loads(gpt35['questions'][idx]), indent=4))
print("GPT-4:\n"+json.dumps(json.loads(gpt4['questions'][idx]), indent=4))